In [1]:
import os

import copy
"""
Assignment statements in Python do not copy objects!
they create bindings between a target and an object.
a copy is sometimes needed so one can change one copy without changing the other!
"""
from pathlib import Path

import scipy

import krotov

import numpy as np
import qutip
import matplotlib.pyplot as plt
#import colorsys
#from mpl_toolkits.mplot3d import Axes3D

import qdyn.model
import qdyn.pulse
# QDYN-pylib is a Python package for interacting with the Fortran QDYN library and tools
#!make clean

from multiprocessing import Pool

from qutip import tensor,qeye,ket
from qutip.qip.circuit import t_gate

from qutip.qip.operations import berkeley,swap

import subprocess as subp

In [2]:
%matplotlib widget

In [3]:
#final time
τ = 2


#t_rise
tr = (τ/10)

#delta_t
Δt = 0.01

#lambda
λ = 0.008

# Number of time points
n = int(τ/Δt + 1)


<font size="5"> Consider THREE qubits.
$ \hat{H_0} = \sum_{i = 1}^{3} \frac{\omega_{i}}{2} Z^{i} + k_1 X^{1}X^{2} + k_2 Z^{1}Z^{2} + k_3 X^{2}X^{3}$ 
    
Imposed with a time-dependent local control $\hat{H(t)}_\text{1} = u(t)X^{(2)}$

In [4]:
#Target-Gate

# Target
B = berkeley(2)
#X = qutip.operators.sigmax()
E = qeye(2)

# Target
B = berkeley(2)

E = qeye(2)

S = swap(2)

U = tensor(S,E)*tensor(E,B)*tensor(S,E)


Target = U

Target = np.array(Target)
target_gate = qutip.Qobj(Target)





# Initialize List
L = [τ,tr,Δt,λ,target_gate]

In [5]:
# lambda t - "anonymous" function of t

def get_Ham(ν1=5.0, ν2 = 5.3, ν3 = 5.5, T = τ, t_rise = tr): 
    u1 = lambda t, args: krotov.shapes.flattop(t, t_start=0.0, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*(6/10*np.cos(ν1*t*2*np.pi))
    u2 = lambda t, args: krotov.shapes.flattop(t, t_start=0.0, t_stop=T, t_rise = t_rise, t_fall = t_rise, func='blackman')*(6/10*np.cos(ν2*t*2*np.pi))
    
    
    Z = qutip.operators.sigmaz()
    X = qutip.operators.sigmax()
    E = qeye(2)
    
    X1X2 = tensor(X,X,E)
    X2X3 = tensor(E,X,X)
    Z1Z2 = tensor(Z,Z,E)
    
 
    k1 = 0.112358
    k2 = 0.222222
    k3 = 0.314159
    
    drift = -1/2 * ( ν1*tensor(Z,E,E) + ν2*tensor(E,Z,E) + ν3*tensor(E,E,Z) ) 
    coupl = k1*X1X2 + k2*Z1Z2 + k3*X2X3
    
    H0 = drift + coupl
    H0 = np.array(H0)  # important for dimensionality. Code recognizes the 8 x 
    
    Ham_0 = qutip.Qobj(H0)
    
    H1 = tensor(X,E,E)
    H1 = np.array(H1)
    Ham_1 = qutip.Qobj(H1)
    
    H2 = tensor(E,X,E)
    H2 = np.array(H2)
    Ham_2 = qutip.Qobj(H2)

    
    return [Ham_0, [Ham_1, np.vectorize(u1)], [Ham_2, np.vectorize(u2)] ]


In [6]:
def qdyn_model(
    Ham,
    T, # final_time
    nt, # number of time steps
    basis, # logical basis
    gate, # target gate
    runfolder,
    prop_method="exact",
    J_T_conv=1e-4,
    Δ_J_T_conv = 1e-7,
    iter_stop=200000,
    **pulse_oct_kwargs, # kwargs = keyword arguments
):
    """ Create qdyn-model

    """
    runfolder = Path(runfolder) # path to runfolder
    ! mkdir $runfolder # bash command 
    tgrid = qdyn.pulse.pulse_tgrid(T=T, nt=nt, t0=0)
      # For technical reasons, QDYN stores the pulses on the intervals of the time grid.
    # That means, the time grid for the pulse needs to be shifted
    #by dt/2 with respect to the time grid used for the propagation
    
    model = qdyn.model.LevelModel()
    # 
    for H in Ham:
        if isinstance(H, list):
            model.add_ham(
                H[0],
                pulse=qdyn.pulse.Pulse(
                    tgrid,
                    amplitude=H[1](tgrid, None),
                    time_unit="ns",
                    ampl_unit="GHz",
                    config_attribs={
                        **pulse_oct_kwargs
                    }, # Additional config data, for when generating a QDYN config
                    # file section describing the pulse (e.g. {'oct_shape': 'flattop', 't_rise': '10_ns'})
                ),
                op_unit="iu",
            )
        else:
            model.add_ham(H, op_unit="GHz")
            
    qdyn.io.write_cmplx_array(
        qutip.Qobj(gate).full().flatten(order="F"), str(runfolder/"gate.dat")
    )
    model.set_propagation(
        T, nt, time_unit="ns", prop_method=prop_method
    )
    model.set_oct("krotovpk", J_T_conv, 5000, iter_stop=iter_stop, continue_=False, delta_J_T_conv=Δ_J_T_conv)
    
    # Observables
    Z = qutip.operators.sigmaz()
    E = qeye(2)
    
    Z1 = qutip.Qobj(np.array(tensor(Z,E,E)))
    Z2 = qutip.Qobj(np.array(tensor(E,Z,E)))
    Z3 = qutip.Qobj(np.array(tensor(E,E,Z)))
    



    model.add_observable(Z1, "observable.dat", exp_unit="GHz", time_unit="ns", col_label = "1" )
    model.add_observable(Z2, "observable.dat", exp_unit="GHz", time_unit="ns", col_label = "2" )
    model.add_observable(Z3, "observable.dat", exp_unit="GHz", time_unit="ns", col_label = "3" )
    
    
    
    
    for i, b in enumerate(basis):
        model.add_state(b, f"basis{i}")
        

    Path(runfolder).mkdir(exist_ok=True, parents=True)
    model.write_to_runfolder(str(runfolder))

In [7]:
def krotov_routine(L):
    
    [τ,tr,Δt,λ,target_gate] = L

    pulse_oct_kwargs = dict(oct_lambda_a=λ, oct_shape = "flattop",t_rise=qdyn.units.UnitFloat(tr, "ns"), t_fall=qdyn.units.UnitFloat(tr, "ns") )
    rf_p = "rf"+str(τ)
    qdyn_model(get_Ham(T = τ, t_rise = tr), τ, n,[qutip.basis(8,i) for i in range(0,8)], target_gate, rf_p  , **pulse_oct_kwargs)

    env = {**os.environ, "OMP_NUM_THREADS": "1"}
    with open("rf"+str(τ)+"/the_file_you_want_to_save_the_output_to.txt", "w") as f:
        subp.run(
            [
                "qdyn_optimize",
                f"--gate={rf_p}/gate.dat",
                "--basis=basis0,basis1,basis2,basis3,basis4,basis5,basis6,basis7",
                "--J_T=J_T_sm",
                "--internal-units=GHz_units.txt",
                f"--write-optimized-gate={rf_p}/final_gate.dat",
                f"{rf_p}",
            ],
            
            
            stderr=f, stdout=f, env=env
        )
        subp.run(["pwd"], stderr=f, stdout=f)
    
    
#krotov_routine(L)OMP_NUM_THREADS=1 oct_increase_factor=50, 


In [ ]:
def do_final_times(a, b, c):
    """
    a = final time
    b = decreasing parameter
    c = how often shall b be reduced from a
    """
    return np.arange(a, 0, -b)[:c]


# do_optimization_input: Here, we create c Lists, where each list consists of final_time, t_rise, delta_t, lambda_0 and the target. For each final_time that was created in "do_final_times" exists a list P. Hence, there are c Lists in total! Each list P contains information that is necessary for doing the "single optimization - routine"
def do_optimization_input(L,b,c):
 
    a = L[0]
    P = [[m] + L[1:5] for m in do_final_times(a, b, c)]
    
    return P

#def do_make_rf(L,b,c)

# do_krotov_parallel: The Idea here is to make life easier and safe time, therefore we want to parallelize the optimizations! 
# In the parallelization, we simultaneously perform the krotov algorithm with respect to each of the c Lists 
# More technical, for each of the c Lists that was created in "do_optimization_input", we can perforn an optimization using "single_optimization"

def do_parallel_krotov(L,b,c):
    P = do_optimization_input(L,b,c)    #n number of threads
    with  Pool(len(P)) as p:
        results = p.map(krotov_routine,P)
    
               
do_parallel_krotov(L,1,2)

In [ ]:
t, pr = np.loadtxt("rf4.5/pulse1.dat", unpack=True)
guess_pul = qdyn.pulse.Pulse(t, amplitude=pr, time_unit="ns", ampl_unit="GHz")
guess_pul.plot()

In [ ]:
t, pr, pi = np.loadtxt("rf4.5/pulse1.oct.dat", unpack=True)
pul = qdyn.pulse.Pulse(t, amplitude=pr+1j*pi, time_unit="ns", ampl_unit="GHz")
#fig, ax = plt.subplots()
pul.plot()

In [ ]:
# Plot pulses against their respective time grid
def plot_pulse(pulse, tlist, xlimit=None):
    fig, ax = plt.subplots()
    if callable(pulse):
        pulse = np.array([pulse(t, None) for t in tlist])
    ax.plot(tlist, pulse)
    ax.set_xlabel('time (ns)')
    ax.set_ylabel('GHz')
    if xlimit is not None:
        ax.set_xlim(xlimit)
    plt.show(fig)
    



In [ ]:

def propagate_obs():
    env = {**os.environ, "OMP_NUM_THREADS": "1"}
    with open("another_output_to.txt", "w") as f:
        subp.run(
            [
                "qdyn_prop_traj", f"--write-all-states=AllStates.dat",
                f"--use-oct-pulses",
                f"--time-unit=ns",
                f"--state-label=basis2",
                f"rf10"
            ],
            stderr=f, stdout=f, env=env
        )
        subp.run(["pwd"], stderr=f, stdout=f)
        

        
propagate_obs()

In [ ]:
t, Z1, Z2, Z3 = np.loadtxt("rf10/observable.dat", unpack=True)


In [ ]:
plot_pulse(Z1,t)

In [ ]:
plot_pulse(Z2,t)

In [ ]:
plot_pulse(Z3,t)

In [ ]:
# Final gate

In [ ]:

A = qdyn.io.read_cmplx_array("rf20/final_gate.dat").reshape(8,8)

A2 = qutip.Qobj(A)

z = A2[0,0]

z

ϕ = np.angle(z)

ϕ

phase = np.e**(-1.0j* ϕ)


A3 = phase*A2

A3
